In [34]:
import pandas as pd
import numpy as np

LOAD FILES

In [35]:
demographie = pd.read_csv('../data/data_demographie.csv',sep=';',skiprows=2)
education = pd.read_csv('../data/data_education.csv', sep=';')
immobilier = pd.read_csv('../data/data_immobilier.csv',sep=';',skiprows=2)
entreprise = pd.read_csv('../data/data_entreprise.csv',sep=';',skiprows=2)
travail = pd.read_csv('../data/data_travail.csv',sep=';',skiprows=2)
equipement = pd.read_csv('../data/data_equipements.csv',sep=';',skiprows=2)

/Users/dldk/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


RENAME COLUMNS

In [36]:
new_dem_col = ['code','commune','pop','1_child-25','Indice_vieillissement','density_pop', \
               'evol_pop','pop-15y','pop-25y','pop-64y','pop+65y','fmly_no_child-25', \
               'fmly_2_child-25','fmly_3_child-25','mean_familiy_size']

new_edu_col =  ['code','commune','ratio_no_diploma','ratio_CAP_BEP','ratio_bac','ratio_etude_sup']

new_immo_col =  ['code','commune', 'res_princ_46', 'res_princ', 'res_second', 'log_vacants', 'appt',\
                 'maison','propr', 'locataire', 'hlm']

new_entreprise_col = ['code','commune','nb_crea_ent','nb_ent', "part_0_sal", "part-10sal", "part+10sal", \
                     "part_agri", "part_indus", "part_const", "part_commerce", "part_admin"]

new_travail_col = ['code','commune','part_salarié', 'part_non_salarié', 'part_agri', 'part_arti', \
                   'part_cadre', 'part_employe', 'part_prof_inter', 'part_ouvrier', 'ta1', 'ta2', \
                   'ta3', 'ta_femme', 'ta_homme']

new_equi_col = ['code','commune', 'hypermarché', 'pôle emploi','banques', 'poste', 'epicerie', \
                'boulangerie', 'boucherie', 'maternelle', 'élémentaire', 'collège', 'urgences', \
                'lycee', 'dentiste', 'medecin','infirmier', 'creche', 'kine', 'pharmacie', 'epahd']

demographie.columns = new_dem_col
education.columns = new_edu_col
immobilier.columns = new_immo_col
entreprise.columns = new_entreprise_col
travail.columns = new_travail_col
equipement.columns = new_equi_col

ADD A NEW COLUMN TO DEMOGRAPHIE : SIZE OF THE CITY

In [37]:
conditions = [
    (demographie['pop'] < 2000),
    (demographie['pop'] >= 2000) & (demographie['pop'] < 5000),
    (demographie['pop'] >= 5000) & (demographie['pop'] < 20_000),
    (demographie['pop'] >= 20_000) & (demographie['pop'] < 50_000),
    (demographie['pop'] >= 50_000) & (demographie['pop'] < 200_000),
    (demographie['pop'] >= 200_000)]
choices = ['Village', 'Bourg', 'Petit ville','Ville moyenne','Grande ville','Métropole']
demographie['type_agglo'] = np.select(conditions, choices, default='Ville')

CHECK NA, REMOVE SOME LINES, FILL OTHERS ...

In [38]:
def ratio_na(col):
    ratio = len(col[col.isna()])/len(col)
    return ratio

def print_na(df):
    for col in df:
        print('Column ', df[col].name, 'has ',ratio_na(df[col]),'% of missing value')

In [39]:
# We remove cities with missing values : it appears that either this town are located in Mayotte
# Either these towns have been destroyed during WWII but still exist in INSEE database
# Or these town are really small (less < 10 habs) and have a lot of missing values 
demographie = demographie[~demographie['1_child-25'].isna()]
education = education[~education["ratio_bac"].isna()]
immobilier = immobilier[~immobilier["res_princ_46"].isna()]
entreprise = entreprise[~entreprise["part_0_sal"].isna()]
travail = travail[~travail["ta2"].isna()]

In [40]:
mean_old_minus_50 = demographie['Indice_vieillissement'][demographie['pop']<50].mean()
demographie['Indice_vieillissement'].fillna(mean_old_minus_50, inplace=True)

In [41]:
# Specific to entreprise, we put 0 default values to some relative value 
#(which are Nan because INSEE try to divide by 0)
# Idem for travail (lot of NAN for this one)
entreprise = entreprise.fillna(0)
travail = travail.fillna(0)

MERGE

In [42]:
education = education.drop(columns=["commune"])
immobilier = immobilier.drop(columns=["commune"])
entreprise = entreprise.drop(columns=["commune"])
travail = travail.drop(columns=["commune"])
equipement = equipement.drop(columns=["commune"])

In [43]:
INSEE_datasets = demographie.merge(education, how='inner', left_on='code', right_on='code') \
                            .merge(immobilier, how='inner', left_on='code', right_on='code') \
                            .merge(entreprise, how='inner', left_on='code', right_on='code') \
                            .merge(travail, how='inner', left_on='code', right_on='code') \
                            .merge(equipement, how='inner', left_on='code', right_on='code')

In [44]:
for col in INSEE_datasets.columns:
    if col != "code" and col != "commune" and col != "type_agglo":
        INSEE_datasets[col] = INSEE_datasets[col].astype(float)

In [45]:
print_na(INSEE_datasets)

Column  code has  0.0 % of missing value
Column  commune has  0.0 % of missing value
Column  pop has  0.0 % of missing value
Column  1_child-25 has  0.0 % of missing value
Column  Indice_vieillissement has  0.0 % of missing value
Column  density_pop has  0.0 % of missing value
Column  evol_pop has  0.0 % of missing value
Column  pop-15y has  0.0 % of missing value
Column  pop-25y has  0.0 % of missing value
Column  pop-64y has  0.0 % of missing value
Column  pop+65y has  0.0 % of missing value
Column  fmly_no_child-25 has  0.0 % of missing value
Column  fmly_2_child-25 has  0.0 % of missing value
Column  fmly_3_child-25 has  0.0 % of missing value
Column  mean_familiy_size has  0.0 % of missing value
Column  type_agglo has  0.0 % of missing value
Column  ratio_no_diploma has  0.0 % of missing value
Column  ratio_CAP_BEP has  0.0 % of missing value
Column  ratio_bac has  0.0 % of missing value
Column  ratio_etude_sup has  0.0 % of missing value
Column  res_princ_46 has  0.0 % of missing

In [46]:
INSEE_datasets.to_csv('../data/data_INSEE_clean.csv', index=False, sep=';')